# Fine-Tuning LLMs

In this exercise, you will fine-tune the [Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model for enhanced dialogue summarization. You will first explore a full fine-tuning approach and evaluate the results with ROUGE metrics. Then you will perform Parameter-Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.

## 1. Set up Dependencies and Load Dataset and LLM

In [1]:
!pip install datasets evaluate rouge_score peft -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00


In [2]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import load_dataset

In [3]:
dataset = load_dataset('knkarthick/dialogsum')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Load the pre-trained [Flan-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of Flan-T5. Setting `torch_dtype=torch.bfloat16` specifies the data type to be used by this model, which can reduce GPU memory usage since `bfloat16` uses half as much memory per number compared to `float32`, the default precision for most models.

In [4]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

## 2. Test the Model with Zero-Shot Inferencing

Test the model with zero-shot inference.

In [ ]:
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(dash_line)
print(f'INPUT PROMPT:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_summary}\n')

You can see that the model struggles to summarize the dialogue compared to the baseline summary, and simply repeats the first sentence from the dialogue.

## 3. Perform Full Fine-Tuning

### 3.1 Preprocess the Dataset

You need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Summarize the following conversation.`, and to the start of the summary with `Summary:` as follows:

Training prompt (dialogue):
```
Summarize the following conversation.
Alice: This is her part of the conversation.
Bob: This is his part of the conversation.    
Summary:
```

Training response (summary):
```
Both Alice and Bob participated in the conversation.
```

**Exercise**: Write a function to tokenize a batch of examples from the dialogue dataset. The function should concatentate the dialogues with the predefined prompt, tokenize them along with their summaries, and define the tokenized summaries as the labels.

In [5]:
def tokenize(examples):
    ### WRITE YOUR CODE HERE

    ## Create the input prompts by concatenating dialogue with the predefined instruction
    inputs = []
    for dialogue in examples['dialogue']:
        prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:"
        inputs.append(prompt)

    # Tokenize the input prompts

    #Truncate sequences that are too long
    #Pad shorter sequences to match the longest in the batch
    #return_tensors=None: Return as lists rather than tensors (required for dataset mapping)
    model_inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors=None)

    # Tokenize the target summaries (labels)
    targets = tokenizer(examples['summary'], truncation=True, padding=True, return_tensors=None)

    # Set the labels to the tokenized summaries

    # labels are the target outputs that want the model to learn to generate. They represent the "correct answers" during training.
    model_inputs['labels'] = targets['input_ids']


    return model_inputs

In [6]:
## Apply preprocessing to entire dataset
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

### 3.2 Fine-Tune the Model

**Exercise**: Utilize the Hugging Face Trainer API for training the model on the preprocessed dataset. Define the training arguments, a data collator, and create a `Seq2SeqTrainer` instance. Train the model for one epoch.

In [7]:
### WRITE YOUR CODE HERE

training_args = Seq2SeqTrainingArguments(
    output_dir='./flan-t5-base-dialogsum',          # Directory to save model checkpoints
    num_train_epochs=1,                             # Train for 1 epoch as specified
    per_device_train_batch_size=8,                  # Batch size per device
    per_device_eval_batch_size=8,                   # Evaluation batch size
    warmup_steps=500,                               # Number of warmup steps
    weight_decay=0.01,                              # regularization technique that adds a penalty term to the loss function to prevent overfitting by penalty for large weights
    logging_dir='./logs',                           # Directory for storing logs
    logging_steps=100,                              # Log every 100 steps
    eval_strategy="epoch",                    # Evaluate at the end of each epoch
    save_strategy="epoch",                          # Save checkpoint at the end of each epoch
    predict_with_generate=True,                     # Use generate for evaluation
    bf16=True,                                      # Use BFloat16 mixed precision training (compatible with bfloat16 model)
    fp16=False,                                     # Disable FP16 since we're using BF16
    push_to_hub=False,                              # Don't push to hub
    report_to=[],
)

## Create data collator

# Handles padding and batching for sequence-to-sequence tasks, Creates attention masks to ignore padding tokens
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=original_model,
    padding=True,
    return_tensors='pt'
)

# create trainer
trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)




/tmp/ipython-input-970557208.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Training a fully fine-tuned version of the model should take about 10 minutes on a Google Colab GPU machine.

In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.520000,0.788586


TrainOutput(global_step=1558, training_loss=9.038452339417208, metrics={'train_runtime': 3108.0589, 'train_samples_per_second': 4.009, 'train_steps_per_second': 0.501, 'total_flos': 8532076611502080.0, 'train_loss': 9.038452339417208, 'epoch': 1.0})

Save the model to a local folder:

In [9]:
model_path = './flan-t5-base-dialogsum-checkpoint'

original_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./flan-t5-base-dialogsum-checkpoint/tokenizer_config.json',
 './flan-t5-base-dialogsum-checkpoint/special_tokens_map.json',
 './flan-t5-base-dialogsum-checkpoint/spiece.model',
 './flan-t5-base-dialogsum-checkpoint/added_tokens.json',
 './flan-t5-base-dialogsum-checkpoint/tokenizer.json')

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

In [10]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-checkpoint',
                                                       torch_dtype=torch.bfloat16)

Reload the original Flan-T5-base model:

In [11]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 3.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Section 2, using the original model and the fully fine-tuned model.

In [12]:
### WRITE YOUR CODE HERE
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

# Create the prompt
prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"

#Generate summary with original model
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

#Generate summary with fine-tuned (instruct) model
output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
instruct_model_summary = tokenizer.decode(output, skip_special_tokens=True)

# Display results for comparison
print(dash_line)
print(f'INPUT DIALOGUE:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'ORIGINAL MODEL SUMMARY:\n{original_model_summary}')
print(dash_line)
print(f'FINE-TUNED MODEL SUMMARY:\n{instruct_model_summary}')


----------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
-----------------------------------------------------

The fine-tuned model is able to create a much better summary of the dialogue compared to the original model.

### 3.4 Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [13]:
rouge = evaluate.load('rouge')

**Exercise**: Generate the outputs for a sample of the test set with the fine-tuned model (use only the first 10 dialogues and summaries to save time).

In [15]:
### WRITE YOUR CODE HERE
# Generate summaries for the first 10 test examples
dialogues = dataset['test']['dialogue'][:10]
human_baseline_summaries = dataset['test']['summary'][:10]

# Generate summaries with original model
original_model_summaries = []
for dialogue in dialogues:
    prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
    inputs = tokenizer(prompt, return_tensors='pt') # Tokenizer converts text to tokens:
    output = original_model.generate(inputs['input_ids'],max_new_tokens=50)[0] #Model generates from token IDs, the generate() method needs the actual token IDs as input
    summary = tokenizer.decode(output, skip_special_tokens=True) #tokenizer is a tool for converting between text and token
    original_model_summaries.append(summary)

# Generate summaries with fine-tuned (instruct) model
instruct_model_summaries = []
for dialogue in dialogues:
    prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
    inputs = tokenizer(prompt, return_tensors='pt')
    output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    summary = tokenizer.decode(output, skip_special_tokens=True)
    instruct_model_summaries.append(summary)


Evaluate the models computing ROUGE metrics:

In [16]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)]
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)]
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.2200291525872921), 'rouge2': np.float64(0.09872463768115944), 'rougeL': np.float64(0.2048122722454118), 'rougeLsum': np.float64(0.20783402073809054)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.2513271655472785), 'rouge2': np.float64(0.06412202933942064), 'rougeL': np.float64(0.22475869678331475), 'rougeLsum': np.float64(0.22463160160748125)}


The results show substantial improvement in all ROUGE metrics:

In [17]:
print("Absolute percentage improvement of the instruct model over the original model:")

for key in instruct_model_results:
    improvement = instruct_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the instruct model over the original model:
rouge1: 3.13%
rouge2: -3.46%
rougeL: 1.99%
rougeLsum: 1.68%


## 4. Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** instead of "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning, with comparable evaluation results as you will see soon.

One of the most popular PEFT methods is **Low-Rank Adaptation (LoRA)**, which  introduces low-rank matrices to adapt the LLM with minimal additional parameters. In most cases, when someone says PEFT, they typically mean LoRA.  After fine-tuning for a specific task with LoRA, the result is that the original LLM remains unchanged and a newly-trained "LoRA adapter" emerges. This LoRA adapter is much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

At inference time, the LoRA adapter is reunited and combined with its original LLM to serve the inference request. The benefit is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

### 4.1 Setup the LoRA model for Fine-Tuning

You first need to define the configuration of the LoRA model. Have a look at the configuration below. The key configuration element to adjust is the rank (`r`) of the adapter, which influences its capacity and complexity. Experiment with various ranks, such as 8, 16, or 32, and see how they affect the results.

In [18]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
)

Add LoRA adapter layers/parameters to the original LLM to be trained:

In [19]:
peft_model = get_peft_model(original_model, lora_config)

The number of trainable model parameters in the LoRA model is:

In [20]:
peft_model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4093


### 4.2 Train the LoRA Adapter

**Exercise**: Define training arguments and create a `Seq2SeqTrainer` instance for the LoRA model. Use a higher learning rate than full fine-tuning (e.g., `1e-3`).

In [21]:
### WRITE YOUR CODE HERE
# Define training arguments for PEFT with higher learning rate
peft_training_args= Seq2SeqTrainingArguments(
    output_dir='./flan-t5-base-dialogsum-lora',     # Directory to save model checkpoints
    num_train_epochs=1,                             # Train for 1 epoch
    per_device_train_batch_size=8,                  # Batch size per device
    per_device_eval_batch_size=8,                   # Evaluation batch size
    warmup_steps=500,                               # Number of warmup steps
    weight_decay=0.01,                              # Weight decay for regularization
    logging_dir='./logs-peft',                      # Directory for storing logs
    logging_steps=100,                              # Log every 100 steps
    learning_rate=1e-3,                             # Higher learning rate for PEFT (1e-3 vs default ~5e-5)
    eval_strategy="epoch",                          # Evaluate at the end of each epoch
    save_strategy="epoch",                          # Save checkpoint at the end of each epoch
    predict_with_generate=True,                     # Use generate for evaluation
    bf16=True,                                      # Use BFloat16 mixed precision training
    fp16=False,                                     # Disable FP16 since we're using BF16
    push_to_hub=False,                              # Don't push to hub
    report_to=[],
)

# Create data collator for PEFT
peft_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=peft_model,
    padding=True,
    return_tensors='pt'
)

# create PEFT trainer
peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=peft_data_collator,
)


/tmp/ipython-input-3849605010.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  peft_trainer = Seq2SeqTrainer(


Train the PEFT adapter. Training should take about 6 minutes on a Google Colab GPU machine.

In [22]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss
1,0.322600,0.381361


TrainOutput(global_step=1558, training_loss=2.450194215590902, metrics={'train_runtime': 2531.3466, 'train_samples_per_second': 4.922, 'train_steps_per_second': 0.615, 'total_flos': 8667537195663360.0, 'train_loss': 2.450194215590902, 'epoch': 1.0})

Save the model to a local folder:

In [23]:
peft_model.save_pretrained('./flan-t5-base-dialogsum-lora')

Load the PEFT model:

In [24]:
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer

peft_model = AutoModelForSeq2SeqLM.from_pretrained('./flan-t5-base-dialogsum-lora')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

Reload the original Flan-T5-base model:

In [25]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)

### 4.3 Evaluate the Model Qualitatively (Human Evaluation)

**Exercise**: Make inferences for the same example as in Sections 2 and 3, using the original model, the fully fine-tuned model and the PEFT model.

In [26]:
### WRITE YOUR CODE HERE
index = 42
dash_line = '-' * 100

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']


# create the prompt
prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"

# Generate summary with original model
inputs = tokenizer(prompt, return_tensors='pt')
output = original_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
original_model_summary = tokenizer.decode(output, skip_special_tokens=True)

# Generate summary with fine-tuned (instruct) model
output = instruct_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
instruct_model_summary = tokenizer.decode(output, skip_special_tokens=True)

# generate summary with PEFT model
output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
peft_model_summary = tokenizer.decode(output, skip_special_tokens=True)

print(f'INPUT DIALOGUE:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'ORIGINAL MODEL SUMMARY:\n{original_model_summary}')
print(dash_line)
print(f'FINE-TUNED MODEL SUMMARY:\n{instruct_model_summary}')
print(dash_line)
print(f'PEFT MODEL SUMMARY:\n{peft_model_summary}')


INPUT DIALOGUE:
#Person1#: I don't know how to adjust my life. Would you give me a piece of advice?
#Person2#: You look a bit pale, don't you?
#Person1#: Yes, I can't sleep well every night.
#Person2#: You should get plenty of sleep.
#Person1#: I drink a lot of wine.
#Person2#: If I were you, I wouldn't drink too much.
#Person1#: I often feel so tired.
#Person2#: You better do some exercise every morning.
#Person1#: I sometimes find the shadow of death in front of me.
#Person2#: Why do you worry about your future? You're very young, and you'll make great contribution to the world. I hope you take my advice.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.
----------------------------------------------------------------------------------------------------
ORIGINAL MODEL SUMMARY:
#Person1#: I'm not sure how t

### 4.4 Evaluate the Model Quantitatively (with ROUGE Metric)

**Exercise**: Generate the outputs for a sample of the test set with the PEFT model (use only the first 10 dialogues and summaries to save time).

In [27]:
### WRITE YOUR CODE HERE
# Generate summaries for the first 10 test examples with PEFT model
dialogues = dataset['test']['dialogue'][:10]
human_baseline_summaries = dataset['test']['summary'][:10]

# Generate summaries with PEFT model
peft_model_summaries = []
for dialogue in dialogues:
    prompt = f"Summarize the following conversation.\n{dialogue}\nSummary:\n"
    inputs = tokenizer(prompt, return_tensors='pt')
    output = peft_model.generate(inputs['input_ids'], max_new_tokens=50)[0]
    summary = tokenizer.decode(output, skip_special_tokens=True)
    peft_model_summaries.append(summary)

print(f"Generated {len(peft_model_summaries)} summaries with PEFT model")



Generated 10 summaries with PEFT model


Compute ROUGE score for this subset of the data.

In [28]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.2200291525872921), 'rouge2': np.float64(0.09872463768115944), 'rougeL': np.float64(0.2048122722454118), 'rougeLsum': np.float64(0.20783402073809054)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.2513271655472785), 'rouge2': np.float64(0.06412202933942064), 'rougeL': np.float64(0.22475869678331475), 'rougeLsum': np.float64(0.22463160160748125)}
PEFT MODEL:
{'rouge1': np.float64(0.394715651129663), 'rouge2': np.float64(0.11040819467010099), 'rougeL': np.float64(0.29264359077939767), 'rougeLsum': np.float64(0.29372832775376034)}


Notice, that PEFT model results are not too bad, while the training process was much easier!

Calculate the improvement of PEFT over the original model:

In [29]:
print("Absolute percentage improvement of the PEFT model over the original model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - original_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the original model:
rouge1: 17.47%
rouge2: 1.17%
rougeL: 8.78%
rougeLsum: 8.59%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [30]:
print("Absolute percentage improvement of the PEFT model over the instruct model:")

for key in peft_model_results:
    improvement = peft_model_results[key] - instruct_model_results[key]
    print(f'{key}: {improvement*100:.2f}%')

Absolute percentage improvement of the PEFT model over the instruct model:
rouge1: 14.34%
rouge2: 4.63%
rougeL: 6.79%
rougeLsum: 6.91%


You can see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. However, the training requires much less computing and memory resources.